In [ ]:
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
import inspect
from tqdm import tqdm
from keras.preprocessing import image
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 32

In [ ]:
seed = 42

In [ ]:
def preprocess_data(batch_size, img_size):
  train_gen = image.ImageDataGenerator(rescale = 1./255.,
                                       validation_split=0.15,
                                       rotation_range=15,  # rotate the image up to 15 degrees
                                       brightness_range = [0.2, 1.0],
                                      )
  train_generator = train_gen.flow_from_directory(
                                      "/content/drive/MyDrive/sve_O - bez augumentacija/Train",
                                      target_size = img_size,
                                      class_mode = "binary",
                                      shuffle = True,
                                      batch_size = batch_size,
                                      subset='training',
                                      seed=seed
                                      )

  val_generator = train_gen.flow_from_directory(
                                      "/content/drive/MyDrive/sve_O - bez augumentacija/Train",
                                      target_size = img_size,
                                      class_mode = "binary",
                                      shuffle = True,
                                      batch_size = batch_size,
                                      subset='validation',
                                      seed=seed
                                      )

  test_datagen = image.ImageDataGenerator(rescale = 1./255.)
  test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/sve_O - bez augumentacija/Test",
                                         target_size = img_size,
                                         class_mode = 'binary',
                                         batch_size=batch_size,
                                         shuffle= False)
  return train_generator,val_generator,test_generator

In [ ]:
train_processed_224, validation_processed_224, test_processed_224 = preprocess_data(batch_size, img_size=[224,224])

Found 135 images belonging to 2 classes.
Found 22 images belonging to 2 classes.
Found 67 images belonging to 2 classes.


In [ ]:
num_train =train_processed_224.samples
num_iterations = int(num_train/batch_size)

In [ ]:
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [ ]:
new_model_dict = {key: model_dictionary[key] for key in model_dictionary  if key in ['ResNet50V2']}

In [ ]:
for model_name, model in tqdm(new_model_dict.items()):
  m_name = model_name
  model1 = model

100%|██████████| 1/1 [00:00<00:00, 1834.78it/s]


In [ ]:
input_shape=(224,224,3)
train_processed = train_processed_224
validation_processed = validation_processed_224
test_processed = test_processed_224

# load the pre-trained model with global average pooling as the last layer and freeze the model weights
pre_trained_model = model1(include_top=False, pooling='avg', input_shape=input_shape)
#pre_trained_model.trainable = False
for layer in pre_trained_model.layers:
   layer.trainable = False
pre_trained_model.summary()

94668760/94668760 [==============================] - 1s 0us/step
Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                        

In [ ]:
for i, layer in enumerate(pre_trained_model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 pool1_pad False
4 pool1_pool False
5 conv2_block1_preact_bn False
6 conv2_block1_preact_relu False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_pad False
11 conv2_block1_2_conv False
12 conv2_block1_2_bn False
13 conv2_block1_2_relu False
14 conv2_block1_0_conv False
15 conv2_block1_3_conv False
16 conv2_block1_out False
17 conv2_block2_preact_bn False
18 conv2_block2_preact_relu False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_pad False
23 conv2_block2_2_conv False
24 conv2_block2_2_bn False
25 conv2_block2_2_relu False
26 conv2_block2_3_conv False
27 conv2_block2_out False
28 conv2_block3_preact_bn False
29 conv2_block3_preact_relu False
30 conv2_block3_1_conv False
31 conv2_block3_1_bn False
32 conv2_block3_1_relu False
33 conv2_block3_2_pad False
34 conv2_block3_2_conv False
35 conv2_block3_2_bn False
36 conv2_bloc

In [ ]:
# custom modifications on top of pre-trained model
clf_model = tf.keras.models.Sequential()
clf_model.add(pre_trained_model)
clf_model.add(tf.keras.layers.BatchNormalization())
clf_model.add(tf.keras.layers.Dropout(0.3))
clf_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
clf_model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
clf_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23575041 (89.93 MB)
Trainable params: 6145 (24.00 KB)
Non-trainable params: 23568896 (89.91 MB)
_________________________________________________________________


In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 5:
        lr *= 0.5
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [ ]:
history = clf_model.fit(train_processed, epochs=10, validation_data=validation_processed,
                            steps_per_epoch=num_iterations, callbacks=[lr_callback])
test_acc = clf_model.evaluate(test_processed)
predictions = clf_model.predict(test_processed)
predicted_classes = np.where(predictions < 0.5, 0, 1)
true_classes = test_processed.classes
class_labels = list(test_processed.class_indices.keys())

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

Epoch 1/10
4/4 [==============================] - 4s 1s/step - loss: 0.2149 - accuracy: 0.9223 - val_loss: 0.2324 - val_accuracy: 0.9545 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 3s 741ms/step - loss: 0.1714 - accuracy: 0.9417 - val_loss: 0.3035 - val_accuracy: 0.8636 - lr: 0.0010
Epoch 3/10
4/4 [==============================] - 4s 1s/step - loss: 0.2136 - accuracy: 0.9219 - val_loss: 0.3202 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 4/10
4/4 [==============================] - 6s 2s/step - loss: 0.1830 - accuracy: 0.9417 - val_loss: 0.2235 - val_accuracy: 0.9545 - lr: 0.0010
Epoch 5/10
4/4 [==============================] - 8s 2s/step - loss: 0.1781 - accuracy: 0.9609 - val_loss: 0.2936 - val_accuracy: 0.7727 - lr: 0.0010
Epoch 6/10
4/4 [==============================] - 4s 1s/step - loss: 0.1684 - accuracy: 0.9612 - val_loss: 0.1973 - val_accuracy: 0.9545 - lr: 0.0010
Epoch 7/10
4/4 [==============================] - 3s 897ms/step - loss: 0.1606 - accuracy: 0.9320

In [ ]:
# Unfreeze the last few layers of the base model
for layer in pre_trained_model.layers[154:]:
    layer.trainable = True

In [ ]:
for i, layer in enumerate(pre_trained_model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 pool1_pad False
4 pool1_pool False
5 conv2_block1_preact_bn False
6 conv2_block1_preact_relu False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_pad False
11 conv2_block1_2_conv False
12 conv2_block1_2_bn False
13 conv2_block1_2_relu False
14 conv2_block1_0_conv False
15 conv2_block1_3_conv False
16 conv2_block1_out False
17 conv2_block2_preact_bn False
18 conv2_block2_preact_relu False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_pad False
23 conv2_block2_2_conv False
24 conv2_block2_2_bn False
25 conv2_block2_2_relu False
26 conv2_block2_3_conv False
27 conv2_block2_out False
28 conv2_block3_preact_bn False
29 conv2_block3_preact_relu False
30 conv2_block3_1_conv False
31 conv2_block3_1_bn False
32 conv2_block3_1_relu False
33 conv2_block3_2_pad False
34 conv2_block3_2_conv False
35 conv2_block3_2_bn False
36 conv2_bloc

In [ ]:
clf_model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = clf_model.fit(train_processed, epochs=100, validation_data=validation_processed,
                            steps_per_epoch=num_iterations,callbacks=[lr_callback])
test_acc = clf_model.evaluate(test_processed)
predictions = clf_model.predict(test_processed)
predicted_classes = np.where(predictions < 0.5, 0, 1)
true_classes = test_processed.classes
class_labels = list(test_processed.class_indices.keys())

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

Epoch 1/100
4/4 [==============================] - 4s 1s/step - loss: 0.0169 - accuracy: 0.9922 - val_loss: 2.1834 - val_accuracy: 0.5455 - lr: 0.0010
Epoch 2/100
4/4 [==============================] - 6s 2s/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 2.1347 - val_accuracy: 0.6364 - lr: 0.0010
Epoch 3/100
4/4 [==============================] - 3s 746ms/step - loss: 0.0165 - accuracy: 0.9903 - val_loss: 1.6380 - val_accuracy: 0.7273 - lr: 0.0010
Epoch 4/100
4/4 [==============================] - 3s 835ms/step - loss: 0.0554 - accuracy: 0.9766 - val_loss: 0.1935 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 5/100
4/4 [==============================] - 4s 1s/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 0.9760 - val_accuracy: 0.8182 - lr: 0.0010
Epoch 6/100
4/4 [==============================] - 4s 882ms/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 0.1720 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 7/100
4/4 [==============================] - 3s 742ms/step - loss: 0.0140 - acc

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy manually
manual_accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Manual Accuracy: {manual_accuracy * 100:.2f}%")

Manual Accuracy: 86.57%


In [ ]:
# Save the model weights
clf_model.save('O_fine_tune_87.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_true=true_classes, y_pred=predicted_classes)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=class_labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()